# Vera C. Rubin Observatory Scheduler Reports

In [ ]:
report_dir = "/sdf/data/rubin/shared/scheduler/reports"
url_base = "https://usdf-rsp-int.slac.stanford.edu/schedview-static-pages"

In [ ]:
import os
from pathlib import Path
import datetime
from IPython.display import HTML, Markdown

import pandas as pd

In [ ]:
report_list = []
for dir_path, dir_names, file_names in os.walk(report_dir):
    this_path = Path(dir_path)
    for file_name in file_names:
        if file_name.endswith(".html"):
            file_path = str(this_path.joinpath(file_name))
            relative_path = file_path[len(report_dir) + 1 :]
            file_parts = relative_path.split("/")
            if len(file_parts) == 6:
                report, instrument, year_str, month_str, day_str, _ = file_parts
                dayobs = year_str + month_str + day_str
                night_iso = "-".join([year_str, month_str, day_str])
                night_date = datetime.date.fromisoformat(night_iso)
                url = "/".join([url_base, relative_path])
                link = f'<a href="{url}" target="_blank" rel="noopener noreferrer">{report}</a>'
                report_list.append(
                    pd.Series(
                        dict(
                            night=night_date,
                            dayobs=dayobs,
                            report=report,
                            instrument=instrument,
                            url=url,
                            link=link
                        )
                    )
                )

reports = pd.DataFrame(report_list).set_index(['instrument', 'dayobs'])

In [ ]:
report_links = (
    reports
    .reset_index()
    .pivot(
        index=('night', 'instrument'),
        columns='report',
        values='link'
    )
    .sort_values('night', ascending=False)
    .fillna("")
    .reindex(columns=['prenight', 'multiprenight', 'nightsum', 'compareprenight'])
)
display(HTML(report_links.to_html(escape=False)))